<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: Ratas.py 🐁

- Nombre de alumno 1: Geraldyn Pérez
- Nombre de alumno 2: Diego Rojas


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/Geral37/MDS7202.git)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [5]:
df.columns

Index(['id', 'date', 'city', 'lat', 'long', 'pop', 'shop', 'brand',
       'container', 'capacity', 'price', 'quantity'],
      dtype='object')

## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn import set_config
set_config(transform_output="pandas")
from xgboost import XGBRegressor
import joblib 

In [4]:
# Inserte su código acá
# 1. 
seed=1705
train, resto = train_test_split(df, test_size=0.3,random_state=seed)
val, test = train_test_split(resto, test_size=1/3, random_state=seed)

# 2.
def extract_date_features(df):
    df['date'] = pd.to_datetime(df['date'])  
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df

date_transformer = FunctionTransformer(extract_date_features)

# 3.
categorical_cols = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']
numeric_cols = ['lat', 'long', 'pop', 'price']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

preprocessor.set_output(transform='pandas')

# 4.
pipeline = Pipeline(steps=[
    ('date_extractor', date_transformer),   # Extraer día, mes, año
    ('preprocessor', preprocessor),         # Preprocesar numérico y categórico
    ('regressor', DummyRegressor(strategy="mean"))  # Regresor Dummy
])

# 5.
X_train = train.drop(columns='quantity')
y_train = train['quantity']

X_val = val.drop(columns='quantity')
y_val = val['quantity']

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error (MAE) en los datos de validación para DummyRegressor: {mae}')

# 6.
pipeline2 = Pipeline(steps=[
    ('date_extractor', date_transformer),   # Extraer día, mes, año
    ('preprocessor', preprocessor),         # Preprocesar numérico y categórico
    ('regressor', XGBRegressor())  # XGBRegressor como modelo
])

pipeline2.fit(X_train, y_train)


y_pred2 = pipeline2.predict(X_val)


mae = mean_absolute_error(y_val, y_pred2)
print(f'Mean Absolute Error (MAE) en los datos de validación para XGBRegressor: {mae}')

# 7.
joblib.dump(pipeline, 'dummy_model.pkl')
joblib.dump(pipeline2, 'xgb_model.pkl')

C:\Users\geral\AppData\Local\Temp\ipykernel_21656\1374751225.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])
C:\Users\geral\AppData\Local\Temp\ipykernel_21656\1374751225.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


Mean Absolute Error (MAE) en los datos de validación para DummyRegressor: 13590.765980053535
Mean Absolute Error (MAE) en los datos de validación para XGBRegressor: 2512.876669849828


['xgb_model.pkl']

5. 

6. 


## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [8]:
df.columns

Index(['id', 'date', 'city', 'lat', 'long', 'pop', 'shop', 'brand',
       'container', 'capacity', 'price', 'quantity'],
      dtype='object')

In [7]:
# Obtener los nombres de las características procesadas
feature_names = pipeline2.named_steps['preprocessor'].get_feature_names_out()

# Crear la lista de restricciones para cada feature
monotonic_constraints = [0] * len(feature_names)  # Lista con 0's para todas las características

# Aplicar restricción -1 a la característica 'num__price'
if 'num__price' in feature_names:
    monotonic_constraints[feature_names.tolist().index('num__price')] = -1  # Asignar -1 a 'num__price'

# Convertir la lista a formato requerido por XGBoost (string con valores separados por comas)
monotonic_constraints_str = '(' + ','.join(map(str, monotonic_constraints)) + ')'

# Redefinir el pipeline con la restricción aplicada en XGBRegressor
pipeline21 = Pipeline(steps=[
    ('date_extractor', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=monotonic_constraints_str))
])

# Entrenar el pipeline de nuevo
pipeline21.fit(X_train, y_train)

# Predecir y evaluar
y_pred = pipeline21.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print(f'Mean Absolute Error (MAE) en los datos de validación para la relación forzada: {mae}')

Mean Absolute Error (MAE) en los datos de validación para la relación forzada: 2442.134453162661


3. El MAE se reduce de 2512 a 2442 unidades en promedio. Asi que el amigo sí tenía razón.

In [6]:
joblib.dump(pipeline21, 'xgb_model_con_rmn.pkl')

['xgb_model_con_rmn.pkl']

## 3. Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [10]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

# 1.
def objective(trial):
    # Inserte su código acá

    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', min_frequency=min_frequency, sparse_output=False)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )
    preprocessor.set_output(transform='pandas')

    preprocessor.fit(X_train)
    # Obtener nombres de características procesadas
    feature_names = preprocessor.get_feature_names_out()
    
    # Restringir la monotonía en 'price'
    monotonic_constraints = [0] * len(feature_names)
    if 'num__price' in feature_names:
        monotonic_constraints[feature_names.tolist().index('num__price')] = -1

    monotonic_constraints_str = '(' + ','.join(map(str, monotonic_constraints)) + ')'

    # Definir el pipeline
    pipeline3 = Pipeline(steps=[
        ('date_extractor', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(
            monotone_constraints=monotonic_constraints_str,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            max_depth=max_depth,
            max_leaves=max_leaves,
            min_child_weight=min_child_weight,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            random_state=seed
        ))
    ])

    pipeline3.fit(X_train, y_train)
    y_pred = pipeline3.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    trial.set_user_attr("best_pipeline", pipeline3)

    return mae

# 2.
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=seed))
study.optimize(objective, timeout=300)  # Tiempo límite de 5 minutos (300 segundos)

# 3.
# Obtener el mejor pipeline
best_pipeline = study.best_trial.user_attrs["best_pipeline"]

# Reportar resultados
print(f"Número de trials: {len(study.trials)}")
print(f"Mejor MAE en validación: {study.best_value}")
print(f"Mejores hiperparámetros: {study.best_params}")

# 5. Guardar el mejor pipeline en un archivo .pkl
with open('best_pipeline.pkl', 'wb') as f:
    joblib.dump(best_pipeline, f)

Número de trials: 45
Mejor MAE en validación: 2267.431577256508
Mejores hiperparámetros: {'learning_rate': 0.09999979510356036, 'n_estimators': 365, 'max_depth': 6, 'max_leaves': 88, 'min_child_weight': 5, 'reg_alpha': 0.5062686131567655, 'reg_lambda': 0.24164993251711653, 'min_frequency': 0.05360688892585248}


3. El MAE disminuyo de 2442 a 2267, lo que indica que la optimización de hiperparámetros con Optuna mejoró el rendimiento del modelo. Esto se debe a que el ajuste de parámetros permitió un mejor equilibrio entre sobreajuste y subajuste, aumentando la precisión en las predicciones.

4. 
- learning_rate: Controla el tamaño de los pasos que da el modelo en cada iteración para minimizar la pérdida. Valores más pequeños hacen que el modelo aprenda más lento pero más preciso, por lo que hace sentido que el rango sea (0.001, 0.1).

- n_estimators: Define cuántos árboles se entrenarán. Un mayor número de árboles puede mejorar el ajuste, pero también aumenta el riesgo de sobreajuste y el tiempo de entrenamiento. Por lo tanto, usar rango (50, 100) es razonable porque un número bajo de árboles (alrededor de 50) puede ser insuficiente para capturar patrones complejos, mientras que un número más alto (hasta 1000) puede aumentar la capacidad del modelo para aprender, aunque a costa de mayor tiempo de entrenamiento.

- max_depth: Limita la profundidad de cada árbol, controlando cuántas decisiones puede tomar cada uno. Un valor más alto permite que el modelo capture más complejidad, pero puede sobreajustarse.  Un valor bajo (como 3) ayuda a prevenir el sobreajuste en problemas más sencillos, mientras que valores más altos (hasta 10) permiten al modelo captar relaciones más complejas en los datos. Por lo tanto, el rango indicado hace sentido.

- max_leaves: Define el número máximo de hojas en cada árbol. Limitar las hojas ayuda a simplificar el modelo y a controlar la complejidad. En este caso un número bajo de hojas (cercano a 0) fuerza modelos más simples, mientras que un límite de 100 hojas permite a los árboles tener más flexibilidad. Dejarlo en 0 puede permitir que el modelo encuentre automáticamente el mejor valor por lo que el rango indicado hace sentido.

- min_child_weight: Establece el número mínimo de observaciones necesarias en un nodo hoja. Valores más altos reducen la posibilidad de que el modelo aprenda patrones espurios. El rango indicado está bien ajustado para evitar el sobreajuste. Valores más bajos (como 1) permiten nodos más pequeños y detallados, mientras que valores más altos (hasta 5) limitan las divisiones en los árboles, lo que ayuda a evitar que el modelo aprenda patrones irrelevantes.

- reg_alpha y reg_lambda: Son los parámetros de regularización L1 (reg_alpha) y L2 (reg_lambda). Ayudan a prevenir el sobreajuste penalizando los coeficientes excesivamente grandes. El rango indicado hace sentido, ya que, es adecuado para buscar el balance entre evitar el sobreajuste (con valores cercanos a 1) y no regularizar demasiado (cercano a 0).

- min_frequency (en OneHotEncoder): Define el umbral mínimo para incluir una categoría en el codificador. Esto reduce el impacto de categorías raras, eliminando posibles fuentes de ruido. Al elegir un valor entre 0.0 y 1.0, se puede controlar la cantidad de categorías que se consideran, buscando un balance entre mantener la riqueza de los datos y reducir el ruido que podrían generar categorías muy poco frecuentes, por lo que el rango indicado hace sentido.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
#!pip install optuna-integration[xgboost]

In [ ]:
# Inserte su código acá

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>